In [4]:
import qpv2

In [5]:
code = '''
    abort;
    skip;
    [q p] :=0;
    H[p];
    assert Pp[p];
    [ pre: Pp[p], post: P1[p]];
    ( skip _ 0.52 \\otimes H[p] );
    if P1[q] then
        CX[q p]
    else
        while Pm[p] do
            H[p]
        end
    end
'''

In [6]:
res = qpv2.parser.parse(code)
print(res)

abort;
skip;
[q p]:=0;
H[p];
assert Pp[p];
[ pre: Pp[p], post: P1[p] ];
(
  skip _0.52⊗
  H[p] );
if P1[q] then
  CX[q p]
else
  while Pm[p] do    H[p]
  end
end
